<a href="https://colab.research.google.com/github/marendtz/TransformerNER/blob/master/code_prodigy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparations**

In [1]:
import getpass
#get github access token for collab
token = getpass.getpass()

··········


In [14]:
%cd /content/
%rm -r /content/*
%ls -la
%pwd

/content
total 16
drwxr-xr-x 1 root root 4096 Apr 29 09:14 ./
drwxr-xr-x 1 root root 4096 Apr 29 09:02 ../
drwxr-xr-x 4 root root 4096 Apr 27 13:34 .config/


'/content'

In [15]:
# clone github repo
!git clone https://{token}@github.com/marendtz/TransformerNER.git

Cloning into 'TransformerNER'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 109 (delta 55), reused 36 (delta 10), pack-reused 0
Receiving objects: 100% (109/109), 225.07 KiB | 4.25 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [ ]:
%pip install -r /content/TransformerNER/requirements.txt


**Create huggingface dataset from prodigy annotations.jsonl**

In [27]:
notebook_login()

In [37]:
import datasets
from datasets import Features, Sequence
from datasets.features import Value, ClassLabel

import random

from spacy.training import offsets_to_biluo_tags, biluo_to_iob
import spacy

from huggingface_hub import notebook_login, create_repo



In [38]:
# function for splitting text into appropriate sized chunks
def split_text(x, max_length=512):
    length = len(x)
    if length > max_length:
        splits = length // max_length
        y = list()
        [y.append(x[i : i + max_length]) for i in range(0, splits*max_length, max_length)]
        if length % max_length > 0:
            y.append(x[splits*max_length : length])
    else:
        y = list([x])
        
    return y

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [39]:
# we will create a function to convert offset formatted labels to BILUO tags and after to IOB tag
def label_and_fill_table(dataset):
    nlp = spacy.blank("en")

    #print("---------------------------------------------------------------")
    #print("Annotated data given by prodigy:")
    #print(dataset)  

    #print("Created Columns:")
    # add seperate colums for information under "tokens"
    tokenizer_id = [word['tokenizer_id'] for word in dataset['tokens']]
    dataset['tokenizer_id'] = tokenizer_id
    #print("tokenizer_id")
    #print(tokenizer_id)

    tokenizer_word_id = [word['tokenizer_word_id'] for word in dataset['tokens']]
    dataset['tokenizer_word_id'] = tokenizer_word_id
    #print("tokenizer_word_id")
    #print(tokenizer_word_id)

    attention_mask = [word['attention_mask'] for word in dataset['tokens']]
    dataset['attention_mask'] = attention_mask
    #print("attention_mask")
    #print(attention_mask)
    
    tokens = [word['text'] for word in dataset['tokens']]
    dataset['tokens'] = tokens
    #print("tokens")
    #print(tokens)
    
    dataset['ner_tags_biluo'] = offsets_to_biluo_tags(
        nlp(dataset['text']), 
        [(d['start'], d['end'], d['label']) for d in [d for d in (dataset['spans'] or [])]]
        )
    #print('ner_tags_biluo')
    #print(dataset['ner_tags_biluo'])
    
    dataset['ner_tags_iob'] = biluo_to_iob(
        dataset['ner_tags_biluo']
        )       
    #print('ner_tags_iob')
    #print(dataset['ner_tags_iob'])
    if set(dataset['ner_tags_iob'])-set(['O','B-PER','I-PER','B-ORG','I-ORG','B-LOC','I-LOC']):              # handle misaligned tokens even tough the same tokenizer is used: https://github.com/explosion/spaCy/discussions/12247
      dataset['ner_tags_iob'] =  ['O' if x == '-' else x for x in dataset['ner_tags_iob']]


    return dataset

In [19]:
# this class holds a function for tokenizing and aligning labels
class tokenize_and_align_labels():
    def __init__(self, tokenizer, label_all_tokens=False):
        self.tokenizer = tokenizer
        self.label_all_tokens = label_all_tokens
    def tokenize_align(self, examples):
        tokenized_inputs= examples["tokens"]        
        #print("tokenized_inputs")
        #print(tokenized_inputs)
        #print("examples[labels]")
        #print(examples["labels"])
        labels = []
        for idx, label in enumerate(examples["labels"]):
          #print("idx")
          #print(idx)
          #print("label")
          #print(label)
          word_ids = examples["tokenizer_word_id"][idx]
          #print("word_ids")
          #print(word_ids)
          previous_word_idx = None
          label_ids = []          
          for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if self.label_all_tokens else -100)
            previous_word_idx = word_idx
          
          #print("label_ids")
          #print(label_ids)
          labels.append(label_ids)
        
        response = {'input_ids': examples["tokenizer_id"],
                    'labels':labels
                    }
        return response

In [80]:
"""
A class to hold the data and labels for the model.
Can be initialized from a JSONL file and procesed, or from a preprocessed Dataset that has been saved.
"""

# A helper function that returns True if 'jsonl' is found in a string, otherwise False
def is_jsonl(string):
    if 'jsonl' in string:
        return True
    else:
        return False

class NERdataset():
    def __init__(self, data_file, text_col, tokenizer,seed, save = None):
        self.text_col = text_col
        self.data_file = data_file
        self.save = save
        self.ta = tokenize_and_align_labels(tokenizer)
        self.tokenizer = tokenizer
        self.seed = seed

    # Load dataset
        self.load()
    # Process the data (and save if save is not None)
        self.process()


    # Load using datasets.Dataset.from_json if the data is an unprocessed jsonl file, otherwise load from a preprocessed dataset using load_dataset
    def load(self):
        if is_jsonl(self.data_file):
            self.dataset = datasets.Dataset.from_json(self.data_file)
        else:
            print("no json supplied as data_file")
        return self

    def process_jsonl(self):
            
      # create necessary columns from prodigy output + convert offsets to iob format
      self.dataset = self.dataset.map(label_and_fill_table)

      # create information about labels used in the dataset
      #label_list = list(set([it for sl in self.dataset['ner_tags_iob'] for it in sl])) 
      #print("label list")
      #print(label_list)
      #ids = range(len(label_list))
      #self.label2id = dict(zip(label_list, ids))
      # ensure fixed labeling for transparency
      self.label2id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
      #print(self.label2id)
      #self.id2label = dict(zip(ids, label_list))
      # ensure fixed labeling for transparency
      self.id2label = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
      self.num_classes = len(self.label2id)
      self.label_list = self.label2id.keys

      # create label colum with info about labels used
      label_column = []
      for sentence in self.dataset["ner_tags_iob"]:
        label_column.append([self.label2id[tag] for tag in sentence])
      self.dataset = self.dataset.add_column("labels", label_column)

      # remove unneccesary columns
      rm_cols = list(set(self.dataset.column_names) - set([self.text_col, 'tokens','tokenizer_id', 'tokenizer_word_id', 'labels', 'attention_mask']))      
      self.dataset = self.dataset.map(remove_columns=rm_cols)
      
      if self.save:
        self.dataset.save_to_disk("/content/TransformerNER/data/processed_ann.hf")
        

    def create_tokenized_dataset(self): 

      # create new dataset for further processing       
      self.tokenized_dataset = self.dataset

      # new colum that will be used for training
      new_column = [None] * len(self.tokenized_dataset)
      self.tokenized_dataset = self.tokenized_dataset.add_column("input_ids", new_column)
      
      # tokenize and allign all the labels
      self.tokenized_dataset = self.tokenized_dataset.map(self.ta.tokenize_align, batched=True)
           

      # cleanup only only keep columns we need and convert offsets to iob
      rm_cols = list(set(self.tokenized_dataset.column_names) - set(['attention_mask','labels','input_ids']))
      self.tokenized_dataset = self.tokenized_dataset.map(remove_columns=rm_cols)       
      

      # adapt tokens      
      features = Features(
          { 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),  
            'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
            'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)
          }
      )      

      # cleanup only only keep columns we need and convert offsets to iob
      rm_cols = list(set(self.tokenized_dataset.column_names) - set(['attention_mask','labels','input_ids']))
      self.tokenized_dataset = self.tokenized_dataset.map(remove_columns=rm_cols) 
            

      #print(self.tokenized_dataset[0])
      #print(self.tokenized_dataset[1])
      self.tokenized_dataset = self.tokenized_dataset.map(features.encode_example, features=features)
      
      
      if self.save:
        self.tokenized_dataset.save_to_disk("/content/TransformerNER/data/tokenized_ann.hf")        
    
    def create_datasetdict(self): 

      # create    
      intermediate_dataset = self.tokenized_dataset
      # shuffle dataset
      shuffled_dataset = intermediate_dataset.shuffle(seed=self.seed)
      
      # build DataSetDict with train, test, val split
      
      # 90% train, 10% test + validation
      train_testvalid = shuffled_dataset.train_test_split(test_size=0.1)
      # Split the 10% test + valid in half test, half valid
      test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
      
      self.datasetdict = DatasetDict({
          'train': train_testvalid['train'],
          'test': test_valid['test'],
          'valid': test_valid['train']})
      
      if self.save:
        self.datasetdict.save_to_disk("/content/TransformerNER/data/datasetdict.hf")       
        # push the tokenized dataset to the hub
        self.datasetdict.push_to_hub(repo_id="maren-hugg/news_ner_dict", private=True)

   
    # Process the data
    def process(self):
        if is_jsonl(self.data_file):
            self.process_jsonl()
            self.create_tokenized_dataset()  
            self.create_datasetdict()  
        

In [46]:
from collections import defaultdict
import pandas as pd
import numpy as np
from datasets import load_dataset_builder, get_dataset_config_names, load_dataset, DatasetDict

from transformers import AutoTokenizer, TrainingArguments, DataCollatorForTokenClassification, Trainer
import nltk.data
nltk.download('punkt')

import torch.nn as nn
from transformers import XLMRobertaConfig, AutoConfig, XLMRobertaForTokenClassification
from transformers.modeling_outputs import TokenClassifierOutput

from huggingface_hub import notebook_login
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
from seqeval.scheme import IOB2


from torch.nn.functional import cross_entropy
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
ds = load_dataset("xtreme", name=f"PAN-X.en")
tags = ds["train"].features

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [84]:
print(tags)

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None), 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}


In [81]:
xlmr_model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

ann_class = NERdataset(
     data_file="/content/TransformerNER/data/annotations_news.jsonl", 
     text_col="text",
     tokenizer=tokenizer,
     seed = 42,
     save="/content/TransformerNER/data/")

Saving the dataset (0/1 shards):   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

ValueError: ignored

**Load Model and custom DatasetDict and Finetune**

In [57]:
news_ner_dict = load_dataset("maren-hugg/news_ner_dict", use_auth_token=True)

  0%|          | 0/3 [00:00<?, ?it/s]

In [64]:
print(news_ner_dict["train"].features)
label2id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
print(label2id)
id2label = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
print(id2label)
num_classes = len(label2id)
print(num_classes)
label_list = list(label2id.keys())
print(label_list)

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
7
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


In [59]:
# setup for tokenization
xlmr_model_name = "xlm-roberta-base"
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)



In [65]:
# create model
import torch

# passing keyword arguments to the from_pretrained() method overrides default values
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,num_labels=num_classes, id2label=id2label, label2id=label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [66]:
# common approach: results for precision, recall, F1-score
# in NER: all words of an entity need to be predicted correctly in order for a prediction to be counted as correct
# libary seqeval: expects predictions and labels as lists of lists 


# during training we need to convert the outputs of the model into such a list that sequeval expects
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []
  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      # Ignore label IDs = -100
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(ann_class.id2label[label_ids[batch_idx][seq_idx]])
        example_preds.append(ann_class.id2label[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

# helper function for calculation of F1-score for validation set
def compute_metrics(eval_pred):
  y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)
  #print("y_pred")
  #print(y_pred)
  #print("y_true")
  #print(y_true)
  return {"f1": f1_score(y_true, y_pred, average='weighted'),
          "precision": precision_score(y_true, y_pred, average='weighted'),
          "recall": recall_score(y_true, y_pred, average='weighted'),
          "accuracy": accuracy_score(y_true, y_pred),
          "classification_report": classification_report(y_true, y_pred, mode='strict', scheme=IOB2)
          }

In [67]:
len(news_ner_dict["train"])

32

In [69]:
num_epochs = 1
batch_size = 1
logging_steps = len(news_ner_dict["train"]) // batch_size # for ann_class = 0
model_name = f"{xlmr_model_name}-finetuned-ann-en"
training_args = TrainingArguments(output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="steps", eval_steps=1, # "steps": Evaluation is done (and logged) every eval_steps. The evaluation strategy to adopt during training.
                                  save_strategy="steps", save_steps=1e6, 
                                  weight_decay=0.01,
                                  disable_tqdm=False, 
                                  logging_strategy="steps", logging_steps=1,  # "epoch": Logging is done at the end of each epoch. The logging strategy to adopt during training.
                                  push_to_hub=True)

In [71]:
notebook_login()

In [72]:
# pad each input sequence to the largest sequence length in a batch
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [73]:
# init method, to avoid initializing a new model for every Trainer (it loads the untrained model and is called at the beginning of the train() call)
def model_init():
  return (XLMRobertaForTokenClassification
          #.from_pretrained(xlmr_model_name, config=xlmr_config)
          .from_pretrained("maren-hugg/xlm-roberta-base-finetuned-panx-en", config = xlmr_config)   # model needs to be public for that
          .to(device))

In [74]:
trainer = Trainer(model_init=model_init, args=training_args,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  train_dataset=news_ner_dict["train"],
                  eval_dataset=news_ner_dict["test"],
                  tokenizer=xlmr_tokenizer)

Cloning https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-ann-en into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.6k/1.03G [00:00<?, ?B/s]

Download file runs/Apr25_18-25-15_6a1d608efb34/1682447468.8780968/events.out.tfevents.1682447468.6a1d608efb34.…

Download file runs/Apr25_18-25-15_6a1d608efb34/events.out.tfevents.1682447468.6a1d608efb34.149.0:  67%|######6…

Download file training_args.bin: 100%|##########| 2.81k/2.81k [00:00<?, ?B/s]

...

Clean file training_args.bin:  36%|###5      | 1.00k/2.81k [00:00<?, ?B/s]

Download file sentencepiece.bpe.model:   0%|          | 17.0k/4.83M [00:00<?, ?B/s]

Clean file runs/Apr25_18-25-15_6a1d608efb34/events.out.tfevents.1682447468.6a1d608efb34.149.0:   4%|3         …

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.83M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.03G [00:00<?, ?B/s]

In [86]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy,Classification Report
1,0.024100,0.186465,0.000000,0.000000,0.000000,0.875000,precision recall f1-score support LOC 0.00 0.00 0.00 0 ORG 0.00 0.00 0.00 0 micro avg 0.00 0.00 0.00 0 macro avg 0.00 0.00 0.00 0 weighted avg 0.00 0.00 0.00 0


Step,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy,Classification Report
1,0.024100,0.186465,0.000000,0.000000,0.000000,0.875000,precision recall f1-score support LOC 0.00 0.00 0.00 0 ORG 0.00 0.00 0.00 0 micro avg 0.00 0.00 0.00 0 macro avg 0.00 0.00 0.00 0 weighted avg 0.00 0.00 0.00 0


ValueError: ignored

In [ ]:
trainer.push_to_hub(commit_message="Training completed!")